In [1]:
import os
import sys
module_path = '/home/fatemeh/others_projects/pybam'
if module_path not in sys.path:
    sys.path.append(module_path)

In [10]:
import pysam
from collections import defaultdict
from datetime import datetime
import pandas as pd

### Parse the bam file

In [19]:
def readBam(bamFile):
    vals = []
    st = datetime.now()
    cntr = 0
    infoMap = defaultdict()
    samfile = pysam.AlignmentFile(bamFile, 'rb')
#     pysam.AlignedSegment.is_re
    for read in samfile:
        if read.is_unmapped and read.mate_is_unmapped:
            continue
        if read.is_paired:
            if read.is_unmapped:
                continue
            if not read.mate_is_unmapped and read.is_read2:
                continue
            if read.mate_is_unmapped:
                isOrphan = True
        rid = read.qname
        if not read.is_paired:
            print([read.is_paired, rid])
        aligned_tid = read.reference_name
        
        if len(read.get_reference_positions()) == 0:
            print(['zero: ' , rid])
        aligned_lpos = read.get_reference_positions()[0]
        aligned_rpos = read.next_reference_start
        alignment_score = read.get_tag('AS')

        vals += [[rid, aligned_tid, alignment_score, aligned_lpos, aligned_rpos]]
        cntr += 1
        if (cntr % 1000000 == 0):
            print("{} : Read {} lines".format(datetime.now()-st, cntr), end="\r")
    res = pd.DataFrame.from_records(vals)
    res.columns = ['readname', 'txp', 'score', 'lpos', 'rpos']
    return res

In [17]:
# set the sample name and root dir:
root='/mnt/scratch3/fatemeh/pufferfish/outputs'
sample='SRR2431350_1k'

# pufferfish
pf = readBam('{}/puffalign/{}.bam'.format(root, sample))
# bowtie
bt = readBam('{}/bowtie2/{}.bam'.format(root, sample))
idx = bt.groupby(['readname'])['score'].transform(max) == bt['score']
bt = bt[idx]

In [31]:
bt_pf=pd.merge(bt, pf, on = ['readname', 'txp'], how='outer', suffixes=['_bt', '_pf']).fillna(-1)

In [32]:
# pufferfish fp
bt_pf[bt_pf['score_bt'] == -1]

,readname,txp,score_bt,lpos_bt,rpos_bt,score_pf,lpos_pf,rpos_pf
715,SRR2431350.11,ENST00000264033,-1.0,-1.0,-1.0,132.0,8116.0,8071.0
716,SRR2431350.30,ENST00000604882,-1.0,-1.0,-1.0,182.0,254.0,100.0
717,SRR2431350.42,ENST00000432949,-1.0,-1.0,-1.0,194.0,913.0,921.0
718,SRR2431350.91,ENST00000441629,-1.0,-1.0,-1.0,200.0,975.0,977.0
719,SRR2431350.91,ENST00000216281,-1.0,-1.0,-1.0,200.0,1595.0,1597.0
720,SRR2431350.189,ENST00000600213,-1.0,-1.0,-1.0,146.0,228.0,879.0
721,SRR2431350.206,ENST00000391954,-1.0,-1.0,-1.0,133.0,8413.0,8402.0
722,SRR2431350.268,ENST00000429578,-1.0,-1.0,-1.0,162.0,1457.0,1555.0
723,SRR2431350.313,ENST00000284727,-1.0,-1.0,-1.0,176.0,3178.0,3185.0
724,SRR2431350.313,ENST00000409194,-1.0,-1.0,-1.0,176.0,298.0,305.0


In [33]:
# pufferfish fn
bt_pf[bt_pf['score_pf'] == -1]

,readname,txp,score_bt,lpos_bt,rpos_bt,score_pf,lpos_pf,rpos_pf
2,SRR2431350.2,ENST00000495401,0.0,120.0,347.0,-1.0,-1.0,-1.0
7,SRR2431350.11,ENST00000296292,-39.0,4613.0,4568.0,-1.0,-1.0,-1.0
39,SRR2431350.161,ENST00000600213,-53.0,457.0,389.0,-1.0,-1.0,-1.0
40,SRR2431350.162,ENST00000262942,-55.0,1294.0,1384.0,-1.0,-1.0,-1.0
80,SRR2431350.332,ENST00000536684,-20.0,373.0,219.0,-1.0,-1.0,-1.0
81,SRR2431350.332,ENST00000604882,-20.0,374.0,220.0,-1.0,-1.0,-1.0
93,SRR2431350.250,ENST00000549759,0.0,417.0,195.0,-1.0,-1.0,-1.0
100,SRR2431350.36,ENST00000600213,-51.0,601.0,773.0,-1.0,-1.0,-1.0
126,SRR2431350.43,ENST00000392994,0.0,38.0,0.0,-1.0,-1.0,-1.0
130,SRR2431350.116,ENST00000544932,-17.0,88.0,361.0,-1.0,-1.0,-1.0
